<h1>Population generation</h1>

In [53]:
#necessary imports
import numpy as np
import pandas as pd
import scipy.stats
from numpy import random
from scipy.stats import truncnorm
import os.path
import re

In [54]:
#możliwe generowane pola. Generacja opiera się o przecięcie tego zbioru = Ω ze zbiorem generowanym = A ⊂ Ω
populationFields = ["age", "gender", "race", "BMI", "physicalActivity", "otherIllnesses"]
otherIllnesses = ["hypertension","asthma","cancer","diabetes","depression","angina","myocardialInfraction","irritableBowelSyndrome","stroke","migraine"]
ser = pd.Series(otherIllnesses)
comorbid = pd.DataFrame({"otherIllnesses" : ser})
comorbid

,otherIllnesses
0,hypertension
1,asthma
2,cancer
3,diabetes
4,depression
5,angina
6,myocardialInfraction
7,irritableBowelSyndrome
8,stroke
9,migraine


In [55]:
#każda funkcja zwraca ilość == amount danych dla całej populacji
def normal(amount, spec):
    return np.random.normal(spec[0], spec[1], amount)
def choice(amount, spec):
    return np.random.choice(spec[0], amount, True, spec[1])
def minmax (amount, spec):
    X = truncnorm((spec[0] - spec[2]) / spec[3], (spec[1] - spec[2]) / spec[3], loc=spec[2], scale=spec[3])
    return X.rvs(amount)

def weightedSubset(types, vals, amount) :
    ret = pd.DataFrame({"ret" : np.zeros(amount)})
    auxpd = pd.DataFrame()
    for typpe, weight,  in zip(types, vals) :
        genrow = choice(amount, ((typpe, None), (weight, 1-weight)))
        auxpd[typpe] = pd.Series(genrow)
    ret["ret"] = auxpd[auxpd.columns[0:]].apply(lambda x: ','.join(x.dropna()),axis=1)
    
    return ret

#example & check
weightedSubset(["hypertension", "asthma", "cancer", "diabetes"],[0.78, 0.12, 0.01, 0.09], 1000)

,ret
0,hypertension
1,hypertension
2,hypertension
3,
4,hypertension
...,...
995,hypertension
996,hypertension
997,hypertension
998,"hypertension,diabetes"


In [56]:
#przykładowe generowanie populacji z niektórymi cechami

testspec = ["age", "gender", "race", "BMI", "physicalActivity"]
testweights = [[56, 12.5],#age[mean, standard_deviation] - normal dist.
            [["f", "m"],[0.52, 0.48]],#gender[vals, weights] - weighted single choice
            [["white","black","other"],[0.66, 0.22, 0.12]],#race[vals, weights] - weighted single choice
              [12.1212, 44.8364, 27.3397, 4.77216 ], #BMI [min_val, max_val, mean, std]] -beta
              [0, 510, 127.095, 101.904]] #activityMinutes [min_val, max_val, mean, std]] -beta
            #otherIllnesses będziemy modelować z wykorzystaniem metody weighted single choice wybierając
            #ułamki wg występowania choroby w populacji
#raczej przekazanie funkcji zamiast stringów
testdistrmethods = [normal, choice, choice, minmax, minmax]
#testdistrmethods = [normal, choice, choice, normal]
testdata = [testspec, testweights, testdistrmethods]


In [57]:
class Generator :
    #metadata section
    
    #generation spec values
    gendata = []
    data = pd.DataFrame()
    
    def __init__(self, data, metadata) :
        #save metadata
        for attr in metadata :
            #setattr(self, attr, val)
            print(attr)
        self.gendata = data
        return None
    
    def populate(amount, speclist) :
        #przecięcie zboiru generowanych cech i wszystkich możliwych
        columns = set(speclist[0]).intersection(populationFields)

        df = pd.DataFrame()
        for index,vals,method in zip(speclist[0],speclist[1],speclist[2]) :
            df[index] = method(amount, vals)
        self.data = ret
        return
        


In [76]:
class MetaGenerator :
    articleData = list()
    #articleDict = dict()
    def readAllArticles(self) :
        m = re.compile("data[0-9]+.csv")
        dir = os.listdir("./data")
        dir = [ file for file in dir if m.match(file)]
        ret = list()
        for entry in dir :
            df = pd.read_csv(f"./data/{entry}", header=0, index_col=0)
            ret.append(df)
        self.articleData = ret
        return 0
    def __init__(self) :
        
        return None

In [75]:
metagen = MetaGenerator()
metagen.readAllArticles()
metagen.articleData[0]

,Unnamed: 0,0,1,2,3
0,0,age,gender,race,otherIllnesses
1,1,"[70, 6.5]","[['f', 'm'], [0.46, 0.54]]","[['white', 'south asian', 'black', 'other'], [...","[['hypertension', 'asthma', 'cancer', 'diabete..."
2,2,<function normal at 0x15BEC8E0>,<function choice at 0x15BEC6A0>,<function choice at 0x15BEC6A0>,<function choice at 0x15BEC6A0>


In [11]:
def generateComorbidities(types, vals, amount) :
    #example. might be extended later
    return weightedSubset(types, vals, amount)
generateComorbidities(["aaa", "bbb", "ccc", "ddd"],[0.50, 0.40, 0.30, 0.20], 22)

,ret
0,"aaa,bbb"
1,"aaa,ccc"
2,"aaa,bbb"
3,"aaa,bbb,ddd"
4,"aaa,bbb,ddd"
5,
6,"aaa,ddd"
7,aaa
8,"aaa,bbb,ccc"
9,"aaa,ccc"
